In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()


groq_api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002398D621F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002398CA9D450>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [3]:
from langchain_chroma  import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 1, Load chunk and index the contents of the blog to create a retriever
import bs4

loder = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs= loder.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorStore = Chroma.from_documents(documents=splits, embedding=embedding)
retriver = vectorStore.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002399FA737D0>, search_kwargs={})

In [6]:
## Prompt Template 
system_prompt = (
    "you are an  assistant for question-answering task. "
    "Use the following places of retrived context to ans"
    "don't know. use three sentence maximum and keep the"
    "answer cocise"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [7]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [8]:
response=rag_chain.invoke({"input":"what is self-Reflaction"})
response

{'input': 'what is self-Reflaction',
 'context': [Document(id='032aad78-d2f6-4804-8891-44c03de0c7c7', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\n\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.\n\n\nExperiments on AlfWorld Env and HotpotQA. Hallucination is a more common failure than inefficient p

In [9]:
response["answer"]

'Self-reflection in the context of autonomous agents is the ability to analyze and evaluate past actions or decisions, identifying mistakes or inefficient planning, and using this information to improve future actions. It involves reflecting on past experiences and learning from them to refine the planning process.'

In [10]:
rag_chain.invoke({"input":"How do we achive it"})



{'input': 'How do we achive it',
 'context': [Document(id='b648ae4d-d9ce-458a-b323-115399b8eac4', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can 

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


contextualize_q_system_prompt = (
    "Given a chat history and latest user question "
    "which might reference context in the chat history, "
    "formulate a standalon question which can be understand "
    "without the chat history. Do not answer the question, "
    "just reformulate it if needed and otherwise return it as is"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [12]:
history_aware_retriever = create_history_aware_retriever(model, retriver, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002399FA737D0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [13]:
qa_prompt =ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [14]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []
question = " what is self-Reflection"
response = rag_chain.invoke({"input": question, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)

question2 = "Tell me more about it."
response2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})
print(response2['answer'])

Self-reflection is a key component of the Reflexion framework, which enables autonomous agents to learn from their experiences and improve their performance over time. It involves analyzing past actions, identifying mistakes, and adjusting future plans accordingly.

In the context of the Reflexion framework, self-reflection is achieved through a process called "two-shot examples," where a pair of examples are presented to the Large Language Model (LLM): a failed trajectory and an ideal reflection that guides future changes in the plan. These reflections are then added to the agent's working memory, where they can be used as context for querying the LLM in future decision-making processes.


In [16]:
chat_history

[HumanMessage(content=' what is self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-reflection is a process that allows autonomous agents to improve by refining past decisions and correcting mistakes, enabling them to learn from their experiences in real-world tasks. It involves analyzing past actions, identifying errors, and adjusting future plans accordingly. This process is crucial for iterative improvement and adaptation in complex environments.', additional_kwargs={}, response_metadata={})]

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [ ]:
conversational_rag_chain.invoke(
    {
        "input": "what is Task Decomposition?"
    },
    config={
        "configurable": {"session_id": "abc123"}
    }, 
)["answer"]

'Task Decomposition is the process of dividing a complex task into smaller, simpler tasks or steps, making it easier to plan, execute, and manage. It helps to break down a large task into more manageable and organized components.'

In [22]:
conversational_rag_chain.invoke(
    {
        "input": "what are common ways of doing it?"
    },
    config={
        "configurable": {"session_id": "abc123"}
    }, 
)["answer"]

'There are several common ways to achieve task decomposition:\n\n1. Using simple LLM prompting (e.g. "Steps for XYZ.\\n1.")\n2. Task-specific instructions (e.g. "Write a story outline.")\n3. Human inputs and external planning tools (e.g. Planning Domain Definition Language (PDDL))'